Занятие 8. Gradient Boosting Machine ver 2
================

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
import os
os.chdir("C:/Users/USER/Documents/Python/_Python_part_1_Lectures_ver2_ShAD/Shad_AD on Python_1_07/")


Задача.
Нужно предсказать доход человека (больше $50000 или меньше) по результатам его ответов в ходе переписи населения.<br>
https://archive.ics.uci.edu/ml/datasets/Adult

Описание данных:<br>
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

In [4]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?')
# Удаляем колонку education (потому что есть уже закодированная колонка education-num)
df = df.drop('education', axis=1)
# Кодируем отклик в бинарные значения
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})
# удаляем строки с NA значениями
df = df.dropna()

test = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
test = test.drop('education', axis=1)
test['income'] = test['income'].map({' <=50K.': 0, ' >50K.': 1})
test = test.dropna()

In [5]:
df.head()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


Посчитаем распределение классов в отклике.

In [6]:
df['income'].value_counts(normalize=True)

0    0.751078
1    0.248922
Name: income, dtype: float64

Разбиваем датасет на лёрн и тест. Бинаризуем категориальные признаки (эта операция ещё называется one-hot encoding).

In [7]:
X_train = pd.get_dummies(df).drop('income', axis=1)
y_train = df['income']

X_test = pd.get_dummies(test).drop('income', axis=1)
y_test = test['income']

После бинаризации категориальных признаков окажется, что в обучающем множестве содержится на одну колонку больше. Это от того, что в тестовой выборке нет ни одного представителя Голландии в колонке ```native-county```.

In [8]:
print(len(X_train.columns))
print(len(X_test.columns))
# Приводим множество названий колонок к типу set, находим разность двух множеств.
print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

88
87
{'native-country_ Holand-Netherlands'}
set()


Чтобы полечить это, создадим полный список всех колонок (из трейна и из теста) и переиндексируем колонки в соответствии с этим списком. В результате в колонке ```native-country_ Holand-Netherlands``` в тесте появятся NaN-значения, которые мы заменим на нули.

In [9]:
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)

Теперь колонки в трейне и в тесте должны быть идентичными. В том числе идентичным должен быть и порядок колонок. На всякий случай убедимся в этом. 

In [10]:
# Команда all проверяет, все ли значения из входного списка равны True
all(X_train.columns == X_test.columns)

True

Перейдём, наконец, к обучению модели.

### GradientBoostingClassifier

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

model = GradientBoostingClassifier(random_state=42,
                                   # Число деревьев
                                   n_estimators=500,
                                   #  загрязнение измеряем “mse”, “mae” или “friedman_mse” (mse с улучшениями),  
                                   criterion='friedman_mse', 
                                   # Максимальная глубина каждого дерева
                                   #  критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’ (как в AdaBoost)
                                   #  ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.0, 
                                   # Устарело
                                   min_impurity_split=None,
                                   # число узлов в дереве (сравни с RandomForest) !!!!!!!!!!!!!
                                   max_depth=5,
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=5, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # Параметр, уменьшающий переобучение, являющемся весом отдельного дерева.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )


In [12]:
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=5, min_samples_split=10,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)

In [13]:

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.89      0.95      0.92     11360
          1       0.80      0.63      0.70      3700

avg / total       0.86      0.87      0.86     15060



In [14]:
y_pred_train = model.predict(X_train)
print(classification_report(y_train, y_pred_train))

             precision    recall  f1-score   support

          0       0.89      0.95      0.92     22654
          1       0.81      0.63      0.71      7508

avg / total       0.87      0.87      0.86     30162



In [15]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,10773,587
1,1381,2319


In [16]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,21523,1131
1,2773,4735


Посмотрим на важность признаков.

In [17]:
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

,features,importance
78,marital-status_ Married-civ-spouse,0.176956
58,capital-gain,0.162486
77,education-num,0.118393
59,capital-loss,0.103633
0,age,0.103343
27,hours-per-week,0.063297
56,fnlwgt,0.037113
33,occupation_ Exec-managerial,0.030562
86,relationship_ Wife,0.020743
39,occupation_ Other-service,0.020289


Калибровка

In [18]:
from sklearn.calibration import CalibratedClassifierCV

In [19]:
model_sigmoid = CalibratedClassifierCV(model, cv=2, method='sigmoid')
# method : ‘sigmoid’ or ‘isotonic’

In [20]:
# Calibrate probabilities
model_sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=5, min_samples_split=10,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False),
            cv=2, method='sigmoid')

In [24]:
# View calibrated probabilities
model_sigmoid.predict_proba(X_test)[0:11, :]

array([[0.99271753, 0.00728247],
       [0.79575442, 0.20424558],
       [0.75099079, 0.24900921],
       [0.01806447, 0.98193553],
       [0.98883834, 0.01116166],
       [0.23272371, 0.76727629],
       [0.99270348, 0.00729652],
       [0.9489826 , 0.0510174 ],
       [0.08013141, 0.91986859],
       [0.36015792, 0.63984208],
       [0.99080081, 0.00919919]])